### Author: Lendl Lapointe

In [47]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

In [48]:
import os
os.getcwd ()

'C:\\Users\\Lendl\\SDS'

In [49]:
table=pd.read_csv('master_tweets.csv')

In [50]:
#loc = { 'NEGATIVE':'0', 'POSITIVE':'1'}
   
#table['classifier_dilbert.label'] = table['classifier_dilbert.label'].map(loc)

In [51]:
table

,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score
0,0,Tue Nov 29 15:31:21 +0000 2022,0,1.600000e+18,en,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I take 0 action without strategy,2.904332e+09,Massoud,mxssoud,"Ottawa, Ontario",NEGATIVE,0.995961,1 star,0.924041
1,0,Tue Nov 29 15:31:20 +0000 2022,0,1.600000e+18,en,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@GuidoDisalle gm Guido!,1.370000e+18,aaronrferguson.eth (he/him) 🇺🇦 | Слава Україні!,aaronrferguson,"Ottawa, ON Canada",NEGATIVE,0.947551,5 stars,0.498549
2,0,Tue Nov 29 15:31:18 +0000 2022,0,1.600000e+18,en,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@SleuthieGoosie Absolutely agree with you. Not...,2.993299e+08,Middy T,middyt,"Ottawa, Ontario, Canada",POSITIVE,0.984934,1 star,0.283423
3,0,Tue Nov 29 15:31:16 +0000 2022,0,1.600000e+18,en,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@ThisIsKyleR @elonmusk Yes, life must be soooo...",2.798719e+07,GeeGee23,GeeGee23,Ottawa,NEGATIVE,0.997436,1 star,0.315153
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,en,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,0,Fri Dec 16 20:01:46 +0000 2022,0,1.603843e+18,en,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","The US had a legal obligation to implement ""in...",1.159924e+18,Reality Check Marker,RealCheckMarker,"Ottawa, Ontario",NEGATIVE,0.986672,1 star,0.258001
2540,0,Fri Dec 16 20:01:46 +0000 2022,0,1.603843e+18,in,False,"<a href=""http://twitter.com/download/iphone"" r...",SEE YA ……… 👋 https://t.co/nPLgFottir,1.520896e+18,Steve Foget,SteveFoget,"Ottawa, Ontario",NEGATIVE,0.546421,5 stars,0.256201
2541,0,Fri Dec 16 20:01:42 +0000 2022,0,1.603843e+18,en,False,"<a href=""http://twitter.com/#!/download/ipad"" ...","Common people, or commoners for short, were pe...",5.584118e+07,Dan Neuman,dan613,"Ottawa, Canada",NEGATIVE,0.975223,3 stars,0.277259
2542,0,Fri Dec 16 20:01:39 +0000 2022,1,1.603843e+18,en,False,"<a href=""http://twitter.com/download/iphone"" r...",Today is the Day,8.845820e+17,KimberleySkye🇨🇦,KimberCanada49,"Ottawa, Ontario",POSITIVE,0.998020,5 stars,0.505339


In [52]:
table1=table[table['lang']=='en']
table2=table[table['lang']=='fr']
frames = [table1, table2]
test = pd.concat(frames)

In [53]:
test[['location']]=test['user.location'].str.split(',',expand=True)[[0]]
test[['score']]=test['classifier_bert.label'].str.split(' ',expand=True)[[0]]
test['score'].unique()

array(['1', '5', '4', '3', '2', nan], dtype=object)

In [54]:
test[['location']]=test['location'].str.split(' ',expand=True)[[0]]

In [55]:
test['location'].unique()

array(['Ottawa', 'ottawa', 'Ottawa/Gatineau', 'Gatineau', 'Canada',
       'Toronto', 'Ottawa.', 'OTTAWA', 'Toronto/Ottawa', 'Chelsea',
       'Nepean', 'Vanier', 'E-block', 'Downtown', 'Calgary', 'YOW',
       'Ottawa-Gatineau', 'Worldwide', 'Near', '🍁Ottawa', '858', 'Québec',
       'Touraine', nan, 'Hull'], dtype=object)

In [56]:
test2=test[test['location']=='Ottawa']
test3=test[test['location']=='Gatineau']
frame = [test2, test3]
test4 = pd.concat(frame)

In [57]:
test4['location'].unique()

array(['Ottawa', 'Gatineau'], dtype=object)

In [58]:
# Since we have a categorical variable,and we are trying to see if there is any kind of relationship between those two variable. A chi sqaure is preferable in this case.
#That way we will be able to tell if ther is any sort of relatioship between the language for the tweets and and the score.

In [59]:
# Chi square test  with Bert model 
contingency= pd.crosstab(test4['score'], test4['lang'])
contingency

lang,en,fr
score,,
1,793,60
2,50,7
3,215,17
4,167,23
5,609,43


In [60]:
# P-value, significance level 0.05
c, p, dof, expected = chi2_contingency(contingency)
p

0.07023398768214598

In [61]:
# WE  can't reject the null hypothesis. That means there is no relationships between the way  you tweet in  the language the people  use to tweet

In [62]:
def get_asterisks_for_pval(p_val):
    """Receives the p-value and returns asterisks string."""
    if p_val > 0.05:
        p_text = "ns"  # above threshold => not significant
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text

In [63]:
def chisq_and_posthoc_corrected(df):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction)"""
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print(f"Chi2 result of the contingency table: {chi2}, p-value: {p}")
    
    # post-hoc
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    p_vals = []
    print("Significance results:")
    for comb in all_combinations:
        new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
        p_vals.append(p)
        # print(f"For {comb}: {p}")  # uncorrected

    # checking significance
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:5f}; corrected: {corr_p_val:5f} ({get_asterisks_for_pval(p_val)}) reject: {reject}")

In [64]:
chisq_and_posthoc_corrected(contingency)

Chi2 result of the contingency table: 8.658213044813179, p-value: 0.07023398768214598
Significance results:
('1', '2'): p_value: 0.227602; corrected: 0.455203 (ns) reject: False
('1', '3'): p_value: 0.991836; corrected: 1.000000 (ns) reject: False
('1', '4'): p_value: 0.028700; corrected: 0.143498 (*) reject: False
('1', '5'): p_value: 0.817205; corrected: 1.000000 (ns) reject: False
('2', '3'): p_value: 0.343989; corrected: 0.573315 (ns) reject: False
('2', '4'): p_value: 1.000000; corrected: 1.000000 (ns) reject: False
('2', '5'): p_value: 0.180876; corrected: 0.452190 (ns) reject: False
('3', '4'): p_value: 0.133622; corrected: 0.445406 (ns) reject: False
('3', '5'): p_value: 0.818886; corrected: 1.000000 (ns) reject: False
('4', '5'): p_value: 0.019632; corrected: 0.143498 (*) reject: False


In [65]:
# Using a posthoc test with the corected p value shows us there is no difference

In [66]:
# Chi square test  with Dilbert model 
contingence= pd.crosstab(test4['classifier_dilbert.label'], test4['lang'])
contingence

lang,en,fr
classifier_dilbert.label,,
NEGATIVE,1249,137
POSITIVE,585,13


In [67]:
c, p, dof, expected = chi2_contingency(contingence)
p

4.387772361954619e-09

In [68]:
# WE reject the null hypothesis. That means there is  relationships between the way  you tweet in  the language the people  use to tweet

In [69]:
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests

all_combinations = list(combinations(contingence.index, 2))
p_vals = []
for comb in all_combinations:
    # subset df into a dataframe containing only the pair "comb"
    new_df = contingence[(contingence.index == comb[0]) | (contingence.index == comb[1])]
    # running chi2 test
    chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
    p_vals.append(p)

In [70]:
reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]

In [71]:
def chisq_and_posthoc_corrected(df):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction)"""
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print(f"Chi2 result of the contingency table: {chi2}, p-value: {p}")
    
    # post-hoc
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    p_vals = []
    print("Significance results:")
    for comb in all_combinations:
        new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
        p_vals.append(p)
        # print(f"For {comb}: {p}")  # uncorrected

    # checking significance
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:5f}; corrected: {corr_p_val:5f} ({get_asterisks_for_pval(p_val)}) reject: {reject}")

In [72]:
chisq_and_posthoc_corrected(contingence)

Chi2 result of the contingency table: 34.44362646093811, p-value: 4.387772361954619e-09
Significance results:
('NEGATIVE', 'POSITIVE'): p_value: 0.000000; corrected: 0.000000 (****) reject: True


In [73]:
# Using a posthoc test with the corected p value shows us there is a difference in the negative and positive tweets

In [74]:
loc = { 'en':'1', 'fr':'0'}
   
test4['lang'] = test4['lang'].map(loc)
test5=test4.copy()
test4

,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score,location,score
0,0,Tue Nov 29 15:31:21 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I take 0 action without strategy,2.904332e+09,Massoud,mxssoud,"Ottawa, Ontario",NEGATIVE,0.995961,1 star,0.924041,Ottawa,1
1,0,Tue Nov 29 15:31:20 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@GuidoDisalle gm Guido!,1.370000e+18,aaronrferguson.eth (he/him) 🇺🇦 | Слава Україні!,aaronrferguson,"Ottawa, ON Canada",NEGATIVE,0.947551,5 stars,0.498549,Ottawa,5
2,0,Tue Nov 29 15:31:18 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@SleuthieGoosie Absolutely agree with you. Not...,2.993299e+08,Middy T,middyt,"Ottawa, Ontario, Canada",POSITIVE,0.984934,1 star,0.283423,Ottawa,1
3,0,Tue Nov 29 15:31:16 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@ThisIsKyleR @elonmusk Yes, life must be soooo...",2.798719e+07,GeeGee23,GeeGee23,Ottawa,NEGATIVE,0.997436,1 star,0.315153,Ottawa,1
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462,Ottawa,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,Fri Dec 16 14:22:45 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Nous avons vécu de grands moments ensemble, et...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.974261,5 stars,0.353771,Gatineau,5
1886,0,Fri Dec 16 14:22:44 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🫂 Départ de @MarcBureau_, notre Président depu...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.975445,5 stars,0.590412,Gatineau,5
2078,0,Fri Dec 16 19:13:34 +0000 2022,0,1.603831e+18,0,False,"<a href=""http://twitter.com/download/android"" ...",Opération policière en cours : la rue Garneau ...,4.308275e+08,PoliceGatineau,PoliceGatineau,Gatineau,NEGATIVE,0.988661,1 star,0.542539,Gatineau,1
2108,0,Fri Dec 16 19:12:33 +0000 2022,0,1.603830e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Félicitations à Michael Goldbloom, C.M. pour l...",2.437687e+08,Patrimoine canadien,Patrimoinecdn,"Gatineau, Québec",POSITIVE,0.619139,5 stars,0.430385,Gatineau,5


In [75]:
test5

,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score,location,score
0,0,Tue Nov 29 15:31:21 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I take 0 action without strategy,2.904332e+09,Massoud,mxssoud,"Ottawa, Ontario",NEGATIVE,0.995961,1 star,0.924041,Ottawa,1
1,0,Tue Nov 29 15:31:20 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@GuidoDisalle gm Guido!,1.370000e+18,aaronrferguson.eth (he/him) 🇺🇦 | Слава Україні!,aaronrferguson,"Ottawa, ON Canada",NEGATIVE,0.947551,5 stars,0.498549,Ottawa,5
2,0,Tue Nov 29 15:31:18 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@SleuthieGoosie Absolutely agree with you. Not...,2.993299e+08,Middy T,middyt,"Ottawa, Ontario, Canada",POSITIVE,0.984934,1 star,0.283423,Ottawa,1
3,0,Tue Nov 29 15:31:16 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@ThisIsKyleR @elonmusk Yes, life must be soooo...",2.798719e+07,GeeGee23,GeeGee23,Ottawa,NEGATIVE,0.997436,1 star,0.315153,Ottawa,1
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462,Ottawa,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,Fri Dec 16 14:22:45 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Nous avons vécu de grands moments ensemble, et...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.974261,5 stars,0.353771,Gatineau,5
1886,0,Fri Dec 16 14:22:44 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🫂 Départ de @MarcBureau_, notre Président depu...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.975445,5 stars,0.590412,Gatineau,5
2078,0,Fri Dec 16 19:13:34 +0000 2022,0,1.603831e+18,0,False,"<a href=""http://twitter.com/download/android"" ...",Opération policière en cours : la rue Garneau ...,4.308275e+08,PoliceGatineau,PoliceGatineau,Gatineau,NEGATIVE,0.988661,1 star,0.542539,Gatineau,1
2108,0,Fri Dec 16 19:12:33 +0000 2022,0,1.603830e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Félicitations à Michael Goldbloom, C.M. pour l...",2.437687e+08,Patrimoine canadien,Patrimoinecdn,"Gatineau, Québec",POSITIVE,0.619139,5 stars,0.430385,Gatineau,5


In [76]:
loc = { 'Ottawa':'1', 'Gatineau':'0'}

test5['location'] = test5['location'].map(loc)
test5

,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score,location,score
0,0,Tue Nov 29 15:31:21 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I take 0 action without strategy,2.904332e+09,Massoud,mxssoud,"Ottawa, Ontario",NEGATIVE,0.995961,1 star,0.924041,1,1
1,0,Tue Nov 29 15:31:20 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@GuidoDisalle gm Guido!,1.370000e+18,aaronrferguson.eth (he/him) 🇺🇦 | Слава Україні!,aaronrferguson,"Ottawa, ON Canada",NEGATIVE,0.947551,5 stars,0.498549,1,5
2,0,Tue Nov 29 15:31:18 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@SleuthieGoosie Absolutely agree with you. Not...,2.993299e+08,Middy T,middyt,"Ottawa, Ontario, Canada",POSITIVE,0.984934,1 star,0.283423,1,1
3,0,Tue Nov 29 15:31:16 +0000 2022,0,1.600000e+18,1,NaN,"<a href=""http://twitter.com/download/iphone"" r...","@ThisIsKyleR @elonmusk Yes, life must be soooo...",2.798719e+07,GeeGee23,GeeGee23,Ottawa,NEGATIVE,0.997436,1 star,0.315153,1,1
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,Fri Dec 16 14:22:45 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Nous avons vécu de grands moments ensemble, et...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.974261,5 stars,0.353771,0,5
1886,0,Fri Dec 16 14:22:44 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🫂 Départ de @MarcBureau_, notre Président depu...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.975445,5 stars,0.590412,0,5
2078,0,Fri Dec 16 19:13:34 +0000 2022,0,1.603831e+18,0,False,"<a href=""http://twitter.com/download/android"" ...",Opération policière en cours : la rue Garneau ...,4.308275e+08,PoliceGatineau,PoliceGatineau,Gatineau,NEGATIVE,0.988661,1 star,0.542539,0,1
2108,0,Fri Dec 16 19:12:33 +0000 2022,0,1.603830e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Félicitations à Michael Goldbloom, C.M. pour l...",2.437687e+08,Patrimoine canadien,Patrimoinecdn,"Gatineau, Québec",POSITIVE,0.619139,5 stars,0.430385,0,5


In [77]:
test5=test5.dropna()
test5

,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score,location,score
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462,1,5
7,0,Tue Nov 29 15:31:05 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",💥Give back to your community in a BIG way toda...,1.431515e+08,Canoe Kayak Canada,CanoeKayakCAN,"Ottawa, ON",POSITIVE,0.995148,5 stars,0.677167,1,5
8,0,Tue Nov 29 15:31:04 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/iphone"" r...",y’all wouldn’t get it https://t.co/9rzxQRwdO6,1.220000e+18,stunna 🇨🇷,LeGreenCard,"Ottawa, Ontario",NEGATIVE,0.999190,1 star,0.607453,1,1
11,0,Tue Nov 29 15:31:00 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://www.loomly.com/"" rel=""nofollo...",“The problem is that #BC isn’t planning to gen...,2.587749e+09,The Energy Mix,TheEnergyMix,Ottawa,NEGATIVE,0.999067,1 star,0.324010,1,1
12,0,Tue Nov 29 15:30:58 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://trueanthem.com/"" rel=""nofollo...",The Ottawa Police Services Board abruptly ende...,2.121260e+07,CTV Ottawa,ctvottawa,"Ottawa, Ontario",NEGATIVE,0.999141,1 star,0.613773,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,Fri Dec 16 14:22:45 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Nous avons vécu de grands moments ensemble, et...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.974261,5 stars,0.353771,0,5
1886,0,Fri Dec 16 14:22:44 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🫂 Départ de @MarcBureau_, notre Président depu...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.975445,5 stars,0.590412,0,5
2078,0,Fri Dec 16 19:13:34 +0000 2022,0,1.603831e+18,0,False,"<a href=""http://twitter.com/download/android"" ...",Opération policière en cours : la rue Garneau ...,4.308275e+08,PoliceGatineau,PoliceGatineau,Gatineau,NEGATIVE,0.988661,1 star,0.542539,0,1
2108,0,Fri Dec 16 19:12:33 +0000 2022,0,1.603830e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Félicitations à Michael Goldbloom, C.M. pour l...",2.437687e+08,Patrimoine canadien,Patrimoinecdn,"Gatineau, Québec",POSITIVE,0.619139,5 stars,0.430385,0,5


In [78]:
log_regression = LogisticRegression()

In [79]:
loc = { 'NEGATIVE':'1', 'POSITIVE':'0'}

test5['DL'] = test5['classifier_dilbert.label'].map(loc)
test5

C:\Users\Lendl\AppData\Local\Temp\ipykernel_11308\2193494246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test5['DL'] = test5['classifier_dilbert.label'].map(loc)


,Unnamed: 0,time,fav_count,id,lang,possibly_sensitive,source,text,user.id,user.name,user.handle,user.location,classifier_dilbert.label,classifier_dilbert.score,classifier_bert.label,classifier_bert.score,location,score,DL
4,0,Tue Nov 29 15:31:13 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/android"" ...",@funstonpaleo So looking forward to seeing the...,7.250000e+17,Michelle Campbell Mekarski PhD,MichelleCbell,"Ottawa, Ontario",POSITIVE,0.996623,5 stars,0.428462,1,5,0
7,0,Tue Nov 29 15:31:05 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",💥Give back to your community in a BIG way toda...,1.431515e+08,Canoe Kayak Canada,CanoeKayakCAN,"Ottawa, ON",POSITIVE,0.995148,5 stars,0.677167,1,5,0
8,0,Tue Nov 29 15:31:04 +0000 2022,0,1.600000e+18,1,False,"<a href=""http://twitter.com/download/iphone"" r...",y’all wouldn’t get it https://t.co/9rzxQRwdO6,1.220000e+18,stunna 🇨🇷,LeGreenCard,"Ottawa, Ontario",NEGATIVE,0.999190,1 star,0.607453,1,1,1
11,0,Tue Nov 29 15:31:00 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://www.loomly.com/"" rel=""nofollo...",“The problem is that #BC isn’t planning to gen...,2.587749e+09,The Energy Mix,TheEnergyMix,Ottawa,NEGATIVE,0.999067,1 star,0.324010,1,1,1
12,0,Tue Nov 29 15:30:58 +0000 2022,0,1.600000e+18,1,False,"<a href=""https://trueanthem.com/"" rel=""nofollo...",The Ottawa Police Services Board abruptly ende...,2.121260e+07,CTV Ottawa,ctvottawa,"Ottawa, Ontario",NEGATIVE,0.999141,1 star,0.613773,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,0,Fri Dec 16 14:22:45 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Nous avons vécu de grands moments ensemble, et...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.974261,5 stars,0.353771,0,5,1
1886,0,Fri Dec 16 14:22:44 +0000 2022,0,1.603758e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","🫂 Départ de @MarcBureau_, notre Président depu...",1.925231e+08,CREDDO,CREDDO_ca,"Gatineau, Canada",NEGATIVE,0.975445,5 stars,0.590412,0,5,1
2078,0,Fri Dec 16 19:13:34 +0000 2022,0,1.603831e+18,0,False,"<a href=""http://twitter.com/download/android"" ...",Opération policière en cours : la rue Garneau ...,4.308275e+08,PoliceGatineau,PoliceGatineau,Gatineau,NEGATIVE,0.988661,1 star,0.542539,0,1,1
2108,0,Fri Dec 16 19:12:33 +0000 2022,0,1.603830e+18,0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Félicitations à Michael Goldbloom, C.M. pour l...",2.437687e+08,Patrimoine canadien,Patrimoinecdn,"Gatineau, Québec",POSITIVE,0.619139,5 stars,0.430385,0,5,0


In [80]:
X= test5[['lang','location']]
y= test5[['DL']].values.ravel()

Using a ordinal logistic regerssion is convenient since the output is binary variable. That will allow us to give the logg odds 
 of the tweet to negative since negative is equal  between english vs french tweet and also for the location (Ottawa=1,Gatineau=0)

In [81]:
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.3, random_state = 1)

In [83]:
model = LogisticRegression(solver='liblinear', random_state=0).fit(X, y_transformed )

In [84]:
model.coef_

array([[-1.63917357,  0.03966194]])

In [85]:
model.intercept_

array([2.34553417])

People who tweets in english have 0.0004 times the odds of people who tweet in french for their tweets to be negative. In other words less likely.
People who tweets from Ottawa have 3.9  times the odds of people who tweet from Gatineau for their tweets to be negative. In other words more likely.

In [86]:
log_regression.fit(X_train,y_train)

LogisticRegression()

In [87]:
y_pred = log_regression.predict(X_test)

In [88]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[  0,  96],
       [  0, 215]], dtype=int64)

In [89]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6913183279742765
